# Load data

In [3]:
import numpy as np 
from monty.json import MontyDecoder
from monty.serialization import loadfn

data = loadfn('bulk_moduli.json')
structures = data['structures']
targets = np.log10(data['bulk_moduli'])

# Set up model and train

In [4]:
from megnet.data.crystal import CrystalGraph
from megnet.data.graph import GaussianDistance
from megnet.model import megnet_model

model = megnet_model(10, 2, n_blocks=1, lr=1e-2,
                     n1=4, n2=4, n3=4, n_pass=1, n_target=1,
                     graph_convertor=CrystalGraph(),
                     distance_convertor=GaussianDistance(np.linspace(0, 5, 10), 0.5))

In [5]:
model.train(structures, targets, epochs=2)

Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Epoch 1/2
1/1 [==============================] - 3s 3s/step - loss: 0.9982
Epoch 2/2
1/1 [==============================] - 0s 24ms/step - loss: 0.9971


# Prediction

## Predict from structure

In [6]:
from pymatgen import MPRester
mpr = MPRester()

# let's get some structure
structure = mpr.get_structure_by_material_id('mp-2340')

In [7]:
predicted_K = 10**model.predict_structure(structure).ravel()

In [8]:
print('The predicted K for {} is {} GPa'.format(structure.formula, predicted_K[0]))

The predicted K for Na6 O6 is 90.84707641601562 GPa
